In [1]:
from selenium import webdriver
from time import sleep

In [2]:
ch_driver = webdriver.Chrome('C:/Users/user/chromedriver.exe')
url = 'https://www.trivago.co.kr/'
ch_driver.get(url)

In [3]:
search_input = ch_driver.find_element_by_id('horus-querytext')
search_word='도쿄'
search_input.send_keys(search_word)
click_button=ch_driver.find_element_by_class_name('horus-btn-search')
click_button.click()
# 여기서 잠시 기다리게 하고 싶으면 sleep(5) 아니면 implicitliy?

In [7]:
another_button = ch_driver.find_element_by_class_name('df_overlay_close_wrap')
another_button.click()

In [8]:
item_list_section = ch_driver.find_element_by_id('js_itemlist')
# tag_name 'li' 말고 class_name 'js_co_item' 으로 
item_list = item_list_section.find_elements_by_class_name('js_co_item')
len(item_list)

25

In [11]:
hotel_info=item_list[0].find_element_by_class_name('item__details')
hotel_info.text.split('\n')

['팰리스 호텔 도쿄',
 'Hotel',
 '도쿄, Akihabara Station에서부터 1.9 km',
 '9.3',
 '최고 좋음(후기 1221개)']

In [12]:
hotel_name = hotel_info.text.split('\n')[0]
dist_from_station = hotel_info.text.split('\n')[2]
num_rating = float(hotel_info.text.split('\n')[3])
text_rating = hotel_info.text.split('\n')[4].split('(')[0]
review_num = int(hotel_info.text.split('\n')[4].split('(')[1].split(' ')[1][:-2])
print(hotel_name, dist_from_station, num_rating, text_rating, review_num)

팰리스 호텔 도쿄 도쿄, Akihabara Station에서부터 1.9 km 9.3 최고 좋음 1221


In [13]:
item_list[0].find_element_by_class_name('ov-hidden').click()

In [14]:
features=item_list[0].find_element_by_class_name('top-features').find_elements_by_tag_name('li')
# class name 'sl-box__block' 말고 'top-features'로 
len(features)

10

In [15]:
for li in features:
    if li.get_attribute("title") == "이용 가능한 시설":
        print(li.text)
    else:
        print('None')

무료무선인터넷 (로비)
무료무선인터넷 (객실)
수영장
스파
주차장
반려동물 동반 가능
에어컨
레스토랑
호텔 바
헬스클럽


In [16]:
price = item_list[0].find_element_by_class_name('item__best-price').text[:-1]
price

'529,635'

In [25]:
def get_hotel_info():
    one_page_result = []
    
    item_list_section = ch_driver.find_element_by_id('js_itemlist')
    item_list=item_list_section.find_elements_by_class_name('js_co_item')  #len(item_list) = 25
    
    #추가 정보를 확인하기 위해서는 해당 탭이 활성화되어 있어야 한다. 그래서 list comprehensions를 이용해
    #탭들을 모두 클릭하여 활성화 하도록 한 뒤, 아래 정보들을 스크래핑
    [x.find_element_by_class_name('ov-hidden').click() for x in item_list]
    #list comprehensions는 기존의 리스트로 새로운 리스트를 만들고 싶을때 뿐만 아니라,
    #요소의 수행을 반복적으로 하고 싶게 할 때 사용할수도 있다.
    
    for hotel in item_list:
        try: 
            hotel_info=hotel.find_element_by_class_name('item__details')
            hotel_name = hotel_info.text.split('\n')[0]
            dist_from_station = hotel_info.text.split('\n')[2]
            num_rating = float(hotel_info.text.split('\n')[3])
            text_rating = hotel_info.text.split('\n')[4].split('(')[0]
            review_num = int(hotel_info.text.split('\n')[4].split('(')[1].split(' ')[1][:-2])
            price=hotel.find_element_by_class_name('item__best-price').text[:-1]
            
            #one_page_result.append((hotel_name, dist_from_station, num_rating, text_rating, review_num))
          
            features = hotel.find_element_by_class_name('top-features').find_elements_by_tag_name('li')
            
            amenities=[]
            for li in features:
                if li.get_attribute('title') == '이용 가능한 시설':
                    amenities.append(li.text)
                else:
                    amenities.append('None')
            print(hotel_name, dist_from_station, num_rating, text_rating, review_num, amenities)
        except:
            pass
           
    return one_page_result

In [26]:
get_hotel_info()

WebDriverException: Message: unknown error: Element <div class="item__location ov-hidden">...</div> is not clickable at point (453, 20). Other element would receive the click: <div class="mt-win__wrp">...</div>
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)


In [27]:
# api 스크래핑을 할때는 위에서와 달리 훨씬 수월하게 작동하는 것 확인
import requests 

In [31]:
requests.get('https://www.trivago.co.kr/api/v1/_cache/accommodation/2113096/complete-info.json?requestId=v91_01_2_ah_kr_KR_KR').json()["amenities"]

{'topFeatures': [{'title': '무선인터넷 (로비)',
   'iconName': 'wifi',
   'isAvailable': True,
   'isFreeOfCharge': True},
  {'title': '무선인터넷 (객실)',
   'iconName': 'wifi',
   'isAvailable': True,
   'isFreeOfCharge': True},
  {'title': '수영장',
   'iconName': 'pool',
   'isAvailable': True,
   'isFreeOfCharge': False},
  {'title': '스파',
   'iconName': 'spa',
   'isAvailable': True,
   'isFreeOfCharge': False},
  {'title': '주차장',
   'iconName': 'parking',
   'isAvailable': True,
   'isFreeOfCharge': False},
  {'title': '반려동물 동반 가능',
   'iconName': 'pet',
   'isAvailable': True,
   'isFreeOfCharge': False},
  {'title': '에어컨',
   'iconName': 'ac',
   'isAvailable': True,
   'isFreeOfCharge': False},
  {'title': '레스토랑',
   'iconName': 'restaurant',
   'isAvailable': True,
   'isFreeOfCharge': False},
  {'title': '호텔 바',
   'iconName': 'bar',
   'isAvailable': True,
   'isFreeOfCharge': False},
  {'title': '헬스클럽',
   'iconName': 'gym',
   'isAvailable': True,
   'isFreeOfCharge': False}],
 'groupedF

## 프로젝트를 통해 구하고자 한 결과물
트리바고 웹사이트에서 도쿄 호텔 정보를 스크래핑 하고자 했다

## 실패한 이유
Trivago 같은 경우는 스크래핑 하기 까다로운 사이트이기도 하고,

api 스크래핑을 할 수 있음에도 selenium으로 스크래핑을 하고자 했던 점이 잘못된 선택이었다

(--> 코드가 변하지 않았음에도 불구하고
재실행시킬때마다 다른 결과가 프린트됨)

## 느낀 점
api로 스크래핑할 수 있으면 api 스크래핑을 하자